In [17]:
import pandas as pd


data = pd.read_csv('sensor_data.csv')

# Display the first few rows of the dataset
data.head()


,Unnamed: 0,timestamp,sensor_id,location,pressure,temperature,flow_rate,vibration,acoustic,hydrocarbon_gas,anomalized
0,0,2024-07-27 04:48:29.354127,0,Point 1,495.281047,73.269485,2794.499278,0.090707,80.423336,0.474596,1
1,1,2024-07-27 04:58:29.354127,0,Point 1,468.003144,102.498441,2634.015558,0.095375,79.452074,0.572293,0
2,2,2024-07-27 05:08:29.354127,0,Point 1,479.574760,83.298923,2751.042752,0.102437,77.997461,0.584935,1
3,3,2024-07-27 05:18:29.354127,0,Point 1,504.817864,101.664160,2651.854530,0.102302,79.683140,0.438170,1
4,4,2024-07-27 05:28:29.354127,0,Point 1,497.351160,96.842436,2781.375184,0.098828,82.064147,0.499111,1


In [18]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000000 entries, 0 to 2999999
Data columns (total 11 columns):
 #   Column           Dtype  
---  ------           -----  
 0   Unnamed: 0       int64  
 1   timestamp        object 
 2   sensor_id        int64  
 3   location         object 
 4   pressure         float64
 5   temperature      float64
 6   flow_rate        float64
 7   vibration        float64
 8   acoustic         float64
 9   hydrocarbon_gas  float64
 10  anomalized       int64  
dtypes: float64(6), int64(3), object(2)
memory usage: 251.8+ MB


In [19]:
from sklearn.model_selection import train_test_split

# Number of clients for federated learning
num_clients = 10

# Split the dataset into multiple subsets, one for each client
client_data = []
client_size = len(data) // num_clients

for i in range(num_clients):
    start_idx = i * client_size
    end_idx = (i + 1) * client_size if i != num_clients - 1 else len(data)
    client_data.append(data.iloc[start_idx:end_idx])

# Display the size of each client dataset
client_sizes = [len(client) for client in client_data]


In [20]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import numpy as np

# Define a function for federated training
def federated_learning(clients_data, rounds=1):
    # Initialize the global model
    global_model = LogisticRegression(max_iter=100)
    
    for round in range(rounds):
        local_models = []
        local_weights = []
        
        # Train a local model for each client
        for client in clients_data:
            X_client = client.drop(columns=['anomalized', 'Unnamed: 0', 'timestamp', 'sensor_id', 'location'])
            y_client = client['anomalized']
            
            local_model = LogisticRegression(max_iter=100)
            local_model.fit(X_client, y_client)
            local_models.append(local_model)
            local_weights.append(len(client))
        
        # Aggregate the local model coefficients
        avg_coef = np.average([model.coef_ for model in local_models], axis=0, weights=local_weights)
        avg_intercept = np.average([model.intercept_ for model in local_models], axis=0, weights=local_weights)
        
        # Update the global model
        global_model.coef_ = avg_coef
        global_model.intercept_ = avg_intercept
        
    # Set the classes_ attribute to ensure it is defined
    global_model.classes_ = local_models[0].classes_
    
    return global_model

# Train the federated model
federated_model = federated_learning(client_data, rounds=100)

# Evaluate the model on the whole dataset
X_test = data.drop(columns=['anomalized', 'Unnamed: 0', 'timestamp', 'sensor_id', 'location'])
y_test = data['anomalized']
y_pred = federated_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

accuracy


c:\Users\YANSILIYU\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
c:\Users\YANSILIYU\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-lear

KeyboardInterrupt: 